<a href="https://colab.research.google.com/github/econ105/AI/blob/main/%E2%80%9Chf_BART_inference_breakdown_ipynb%E2%80%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How we generate a sentence using seq2seq architecture!

*   清單項目

*   清單項目

*   清單項目
*   清單項目


*   清單項目


*   清單項目


The code is the supplementary material to the story published in NLPiation medium. Follow [the link](https://pub.towardsai.net/a-full-introduction-on-text-summarization-using-deep-learning-with-sample-code-ft-huggingface-d21e0336f50c) for a detailed explanation of the encoder-decoder architecture and code.

> The purpose of this code is to show the flow of the data and basically what is happening under the hood while we are doing the inference. It is the first part of the series where I write about the basics of text summarization task.

## Installing HuggingFace/Importing

Start by installing the Transformers library (by Huggingface) and then import the modules.

In [ ]:
!pip install transformers #install transformers

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration #from transformers import Bart
import torch #import torch

## Load the Model

We should put the model in evaluation mode to get a consistent result after each run and avoid the randomness of layers like dropout. Since the huggingface library load the models in eval() mode by default it is not necesary here.

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn') #initializes a tokenizer for the BART model using the pre-trained weights from the 'facebook/bart-large-cnn' model. The tokenizer is responsible for processing input text and converting it into tokens that can be understood by the BART model.
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn') #initializes the BART model for conditional text generation using the pre-trained weights from the 'facebook/bart-large-cnn' model. This model is capable of generating new text based on a given prompt or input sequence, making it suitable for tasks such as summarization, translation, and text generation.
# model = model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

## Load the Necessary Components

Let's first call the model object to see all the sub-modules and layers. We can find what each layer is called and how many layers are in the network. In the following code I am only calling the encoder component to make the outout smaller. You can run the notebook and uncomment the first print statement to see the full model.

In [ ]:
# print( model )
model.model.encoder

BartEncoder(
  (embed_tokens): Embedding(50264, 1024, padding_idx=1)
  (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
  (layers): ModuleList(
    (0-11): 12 x BartEncoderLayer(
      (self_attn): BartSdpaAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (activation_fn): GELUActivation()
      (fc1): Linear(in_features=1024, out_features=4096, bias=True)
      (fc2): Linear(in_features=4096, out_features=1024, bias=True)
      (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    )
  )
  (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
)

Now, we can use the built-in function to get each componont. The language model head (lm_head) is also necesary as it will generates the last probability output.

In [ ]:
the_encoder = model.get_encoder() #the encoder's job is to process the input data (e.g., a sentence in a source language) and convert it into a set of features or representations that capture the essence of the input. These representations are then passed on to other parts of the model for further processing.
the_decoder = model.get_decoder() #In models that have both an encoder and a decoder (like the transformer model), the decoder takes the representations produced by the encoder and generates the output sequence step by step (e.g., a sentence in the target language for machine translation). The decoder has access to the encoder's output and also to the parts of the output that it has generated so far, using this information to produce the next element in the output sequence.
last_linear_layer = model.lm_head #"LM" stands for language model, and in this context, the "head" typically refers to the final layer or a specific part of the model designed for a particular task (e.g., generating text, classifying text into categories, etc.).

## Tokenize a Sample Article

In [ ]:
ARTICLE_TO_SUMMARIZE = """sebastian vettel is determined to ensure the return of a long-standing ritual at ferrari is not a one-off this season. fresh from ferrari's first victory in 35 grands prix in malaysia 11 days ago, and ending his own 20-race drought, vettel returned to a hero's welcome at the team's factory at maranello last week. the win allowed ferrari to revive a tradition not seen at their base for almost two years since their previous triumph in may 2013 at the spanish grand prix courtesy of fernando alonso. sebastian vettel reflected on his stunning win for ferrari at the malaysian grand prix during the press conference before the weekend's chinese grand prix in shanghai the four-time world champion shares a friendly discussion with mclaren star jenson button four-times world champion vettel said: 'it was a great victory we had in malaysia, great for us as a team, and for myself a very emotional day - my first win with ferrari. 'when i returned to the factory on wednesday, to see all the people there was quite special. there are a lot of people working there and as you can imagine they were very, very happy. 'the team hadn't won for quite a while, so they enjoyed the fact they had something to celebrate. there were a couple of rituals involved, so it was nice for them to get that feeling again.' asked as to the specific nature of the rituals, vettel replied: 'i was supposed to be there for simulator work anyway, but it was quite nice to receive the welcome after the win. ferrari's vettel and britta roeske arrive at the shanghai circuit along with a ferrari mechanic, vettel caught up with members of his old team red bull on thursday 'all the factory got together for a quick lunch. it was quite nice to have all the people together in one room - it was a big room! - so we were able to celebrate altogether for a bit. 'i also learned when you win with ferrari, at the entry gate, they raise a ferrari flag - and obviously it's been a long time since they last did that. 'some 10 years ago there were a lot of flags, especially at the end of a season, so this flag will stay there for the rest of the year. 'we will, of course, try and put up another one sometime soon.' inside the ferrari garage, vettel shares a discussion with team staff as he looks to build on his sepang win ferrari team principal maurizio arrivabene shares a conversation with vettel at the team's hospitality suite the feeling is that will not happen after this weekend's race in china as the conditions at the shanghai international circuit are expected to suit rivals mercedes. not that vettel believes his success will be a one-off, adding: 'for here and the next races, we should be able to confirm we have a strong package and a strong car. 'we will want to make sure we stay ahead of the people we were ahead of in the first couple of races, but obviously knowing mercedes are in a very, very strong position. 'in general, for the start of a season things can be up and down, and we want to make sure there is quite a lot of ups, not so many downs. 'but it's normal in some races you are more competitive than others. 'we managed to do a very good job in malaysia, but for here and the next races we have to be realistic about we want to achieve.' ferrari mechanics show their joy after vettel won the malaysian grand prix, helping record the team's first formula one win since 2013 at the spanish grand prix"""

In [ ]:
tokenized_input = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, truncation=True, return_tensors='pt')


This line of code involves the use of a tokenizer to prepare an input article for processing by a machine learning model, perhaps for a task like summarization. Let's break down what this code does:
`tokenizer`:** This is an instance of a tokenizer that is used to convert raw text into a format that can be understood by the machine learning model. Tokenizers convert text into tokens (words, subwords, or characters depending on the tokenizer) which are then mapped to numerical IDs. These numerical representations are what models actually process.
[ARTICLE_TO_SUMMARIZE]`:** This suggests that `ARTICLE_TO_SUMMARIZE` is a variable containing the text of an article you want to summarize. By placing it in square brackets, it's turned into a list. Many tokenizers can process multiple inputs at once, so inputs are often provided in list form.

`max_length=1024`:** This parameter sets the maximum length of the tokenized output. If the tokenized version of `ARTICLE_TO_SUMMARIZE` exceeds 1024 tokens, it will be truncated (cut off) to fit this length. This is important because many machine learning models, especially those based on the Transformer architecture, have a fixed maximum input length due to computational constraints.

`truncation=True`:** This explicitly enables truncation. In the context of the `max_length` parameter, setting `truncation=True` means that if the tokenized input is longer than the `max_length`, it will be truncated to meet this constraint. This ensures that the model receives input that adheres to its expected input size limitations.

`return_tensors='pt'`:** This parameter specifies the format of the returned tokenized data. Here `'pt'` stands for PyTorch tensors. This is specifying that the tokenized input should be converted into tensors compatible with PyTorch, a popular deep learning library. Depending on the deep learning library you are using, you might see `'tf'` for TensorFlow tensors or `'np'` for NumPy arrays instead.

The operation of tokenizing the text prepares it in the correct format and size for processing by a model, which could be for tasks such as summarization, classification, translation, etc. When using Transformer-based models (like those available through Hugging Face's Transformers library), preparing the input correctly is key to getting accurate and reliable outputs from the model.

## Encoding the Article

We pass both token_ids and the attention_mask to the model that are generated by the tokenizer.

In [ ]:
input_representation = the_encoder(input_ids = tokenized_input.input_ids,
                                   attention_mask = tokenized_input.attention_mask)
print( input_representation ) # transforming raw input text into a numerical representation, making it ready for further processing by a machine learning model. This transformation involves tokenization, numerical mapping, attention masking, and encoding.

BaseModelOutput(last_hidden_state=tensor([[[ 0.0076,  0.0438,  0.0330,  ...,  0.0049,  0.0072,  0.0013],
         [ 0.2069, -0.5512, -0.0667,  ..., -0.2238, -0.1640,  0.0192],
         [ 0.1963, -0.5009,  0.3967,  ...,  0.0317,  0.0772,  0.1804],
         ...,
         [-0.1127, -0.2903, -0.0103,  ...,  0.0528, -0.1473, -0.0085],
         [ 0.0686, -0.0143, -0.0834,  ..., -0.0261, -0.1815,  0.3171],
         [ 0.0344,  0.4228,  0.0438,  ...,  0.0531,  0.0063, -0.0064]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)


## Decoding the Representation

Now, we use the encoded representation of the input alongside with the start token id to the decoder to predict the first token of the summary. Lastly, pass the decoder's output to the linear layer to get the probability of each token in the vocabulary to be the next predicted token.

In [ ]:
start_token = torch.tensor( [tokenizer.bos_token_id] ).unsqueeze(0)
mask_ids    = torch.tensor( [1] ).unsqueeze(0) #preparing tensors for the start token and a mask ID, both with their dimensions adjusted to fit expected input formats for further processing or model input.

In [ ]:
decoder_output = the_decoder(input_ids=start_token,
                             attention_mask=mask_ids,
                             encoder_hidden_states=input_representation[0],
                             encoder_attention_mask=tokenized_input.attention_mask )

decoder_output = decoder_output.last_hidden_state #performing a decoding operation using a decoder model, which could be part of a sequence-to-sequence architecture, often used in tasks like machine translation, text summarization

In [ ]:
logits = last_linear_layer(decoder_output[0])
predicted_token = logits.argmax(1)
print(predicted_token) #taking the output from a decoder, applying a linear transformation to it to obtain logits, and then identifying the index of the highest scoring logit. This index represents the model's predicted choice.

tensor([1090])


The first predicted token id is **1090**. Let's use the convert_ids_to_tokens() function to see what this id represents in the BART's vocabulary.

In [ ]:
tokenizer.convert_ids_to_tokens(predicted_token) # id representation

['se']

So, it represents 'se' 🤔! We saw how the model generate tokens to form a summary. Let's use the huggingface's generate function to actually generate a summary with more tokens and decode the IDs to readable words.

In [ ]:
summary_ids = model.generate(tokenized_input['input_ids'], max_length=15, early_stopping=True)
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])#generate summaries for some input data, with certain parameters specified such as the maximum length of the summary and enabling early stopping to prevent the generation from becoming too lengthy or diverging.

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1202: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (15). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


['sebastian vettel won the malaysian']


In [16]:
tokenizer.convert_ids_to_tokens( summary_ids[0] )

['</s>',
 '<s>',
 'se',
 'b',
 'ast',
 'ian',
 'Ġve',
 'tt',
 'el',
 'Ġwon',
 'Ġthe',
 'Ġmal',
 'ays',
 'ian',
 '</s>']

'</s>': This token represents the end of a sequence or sentence.
'<s>': This token represents the start of a sequence or sentence.
'se': This token represents the beginning of a word or a subword.
'b': This token represents the beginning of a word or a subword.
'ast': This token represents the continuation of a word or a subword.
'ian': This token represents the continuation of a word or a subword.
'Ġve': This token represents the continuation of a word or a subword. The 'Ġ' character is a special token used by some tokenizers to indicate the beginning of a new word or subword.
'tt': This token represents the continuation of a word or a subword.
'el': This token represents the continuation of a word or a subword.
'Ġwon': This token represents the continuation of a word or a subword. The 'Ġ' character is a special token used by some tokenizers to indicate the beginning of a new word or subword.
'Ġthe': This token represents the continuation of a word or a subword. The 'Ġ' character is a special token used by some tokenizers to indicate the beginning of a new word or subword.
'Ġmal': This token represents the continuation of a word or a subword. The 'Ġ' character is a special token used by some tokenizers to indicate the beginning of a new word or subword.
'ays': This token represents the continuation of a word or a subword.
'ian': This token represents the continuation of a word or a subword.
'</s>': This token represents the end of a sequence or sentence.

In [17]:
summary_ids #summary

tensor([[   2,    0, 1090,  428, 1988,  811, 5030, 5967,  523,  351,    5, 8196,
         4113,  811,    2]])

Interesting, so the word 'se' was the first token to predict the name 'sebastian'! Because the tokenizer break-down multi sylabus words into multiple tokens to reduce the overall vocabulary size.

Read the medium post [here](https://pub.towardsai.net/a-full-introduction-on-text-summarization-using-deep-learning-with-sample-code-ft-huggingface-d21e0336f50c) if you still have question.

Consider following me on Twitter ([@NLPiation](https://twitter.com/NLPiation)) where I am mostly writing about NLP and also is a great place to have discussions.